LZW - Kompression und Expansion von Texten
=====================

Version 1.0, 02.06.2017, W. Conen

In diesem Notebook stellen wir die Arbeitsweise des Lempel-Ziv-Welch-Algorithmus vor, kurz LZW. Dieser Algorithmus war lange Zeit das oft verwendete "Arbeitspferd" bei der Kompression von Daten. Wir orientieren uns weitgehend an der Darstellung in Thomas H. Cormens Buch "Algorithms Unlocked" (MIT Press 2013), dessen Lektüre wir sehr empfehlen können.

Anders, als das bereits vorgestellte Verfahren zur Kompression mit Huffman-Codes, erfordert LZW nur einen Durchgang über die zu komprimierenden Daten (**one-pass Verfahren**, zum Huffman-Verfahren gibt es übrigens auch one-pass Varianten).

Die Grundidee ist recht simpel: der Kompressor gibt eine Folge von Indices aus, die in ein Array von Strings zeigen (eine Art Wörterbuch bzw. Dictionary), die (fast) alle im Text auch vorkommen. Kommen auch relative lange Strings in diesem Array vor, dann können wir auch eine vernünftige Kompression erwarten (weil es günstiger ist, eine vergleichsweise kurze Nummer auszugeben, als den recht langen String, den sie repräsentiert). 

Ein Problem, das entstehen könnte, ist allerdings das Dictionary selbst. Wenn ich dieses Mitübertragen muss zum Expandierer, dann könnte es sein, dass ich gar nichts spare, sondern im Ergebnis sogar mehr Platz brauche! 

Das coole am Algorithmus ist, dass er zwar ein Dictionary benötigt, dieses aber nicht abspeichern muss, sondern bei der Expansion neu und identisch zum Kompressor-Dictionary erzeugen kann.

Schauen wir uns zunächst den Kompressionsalgorithmus an.

Kompression
------------

Zunächst initialisieren wir unser Dictionary mit Einträgen für jedes Zeichen aus dem zugrunde liegenden Zeichensatz (das kann schon ganz schön gross werden, wenn man Unicode nimmt. Für die hier vorgestellte Variante des Algos nehmen wir das einfach mal so hin).

Für unser Beispiel unten reduzieren wir unseren Zeichensatz auf 4 Zeichen, A,C,G,T, die Bausteine der DNS.

Wir wollen nun die Sequenz TATAGATCTTAATATA komprimieren.

Die Idee ist wie folgt: wir lesen nach und nach Zeichen ein. Wir wollen Strings erkennen, die bereits im Dictionary vermerkt sind. Diese sollen möglichst lang sein. Wir versuchen also, erst dann den Index von etwas Bekanntem auszugeben, wenn sich das nicht mehr verlängern lässt, ohne Unbekannt zu werden. Das merken wir, in dem wir das bisher Bekannte, nennen wir es s, um das aktuelle Zeichen, nennen wir es c, ergänzen und schauen, ob das Resultat s+c, also c direkt an s gehängt, bereits im Dictionary vorhanden ist. Wenn ja, machen wir so weiter. Wenn nein, dann geben wir die Indexnummer von s aus, setzen s auf c und merken uns im Dictionary s+c als neuen Eintrag. Das ist alles. Entsprechend arbeitet der Algorithmus unten. Ganz am Ende geben wir noch den Index des verbleibenden s aus (das ist ja sicher im Dictionary! Überlegen Sie, warum!)

Diesen Ablauf wollen wir nun für unseren Beispiel-String als Tabelle nachvollziehen.

<pre>
Schritt |  c | s+c    | s       | Ausgabe | Dictionary:
 Init   |    |        |         |         |  0 : A
        |    |        |         |         |  1 : C
        |    |        |         |         |  2 : G
        |    |        |         |         |  3 : T
        |    |        |  ""     |         | 
   0    |  T |  T     |  T      |         |  
   1    |  A |  TA    |  A      |   3     |  4 : TA
   2    |  T |  AT    |  T      |   0     |  5 : AT
   3    |  A |  TA    |  TA     |         |  
   4    |  G |  TAG   |  G      |   4     |  6 : TAG
   5    |  A |  GA    |  A      |   2     |  7 : GA
   6    |  T |  AT    |  AT     |         |  
   7    |  C |  ATC   |  C      |   5     |  8 : ATC
   8    |  T |  CT    |  T      |   1     |  9 : CT
   9    |  T |  TT    |  T      |   3     |  10 : TT
   10   |  A |  TA    |  TA     |         |  
   11   |  A |  TAA   |  A      |   4     |  11 : TAA
   12   |  T |  AT    |  AT     |         |  
   13   |  A |  ATA   |  A      |   5     |  12 : ATA
   14   |  T |  AT    |  AT     |         |  
   15   |  A |  ATA   |  ATA    |         |  
        |    |        |         |   12    | 
 </pre>   

 
 Der Algo unten ist in Python geschrieben und erzeugt die gerade per Hand gefundene Sequenz von Indices:

In [6]:
def LZW(text):  # Text must be non-empty ;)
    # Init the dictionary
    dic = [ 'A', 'C', 'G', 'T']
    com = []
    
    s = ''
    for c in text:
        if s+c in dic:  # ist s+c schon im Dictionary? 
            s += c      # Ja, dann warten wir, wie es weitergeht
        else:           # Nein? Dann geben wir den Index von s aus, erweitern dic um s+c und setzen s auf c
            com.append(dic.index(s)) # s must always be in dic! dic.index() gives the position of s in dic
            dic.append(s+c)
            s = c
    com.append(dic.index(s))
    return com

LZW("TATAGATCTTAATATA")

[3, 0, 4, 2, 5, 1, 3, 4, 5, 12]

Wir speichern das Dictionary am Emde NICHT ab! Das einzige Wissen, das der Kompressor und der Expandierer gemeinsam haben, ist das Wissen um den Zeichensatz.

Expansion bzw. Dekompression
-----------------------------

Versuchen wir mal, aus dem Ergebnis eben den String zu rekonstruieren, also das Komprimat zu expandieren:

Wir initialisieren zunächst das Dictionary.

0: A, 1: C, 2: G, 3: T

Nun laufen wir über die Sequenz [3, 0, 4, 2, 5, 1, 3, 4, 5, 12] und versuchen, uns quasi "gespiegelt" so zu verhalten, wie der Komprimierer.

Zuerst lesen wir eine 3. Die ist im Dictionary, klar. Wir geben also ein T aus. Wir wissen: der Komprimierer hat bei der Ausgabe der 3 ein Zeichen c in der Hand gehabt, das das T so verlängert, dass T+c nicht im Dictionary war. Diesen String T+c hat er dann ins Dictionary aufgenommmen. Leider wissen wir noch nicht, was c ist, aber das finden wir gleich heraus:

Wir lesen nun eine 0. Das erste Zeichen der Ausgabe, die wir nun erzeugen können, in dem wir im Dictionary an Position 0 nachsehen, muss unser gesuchtes c sein. 0 steht hier für "A", wir können nun also T+A = TA als weiteren Eintrag ins Dictionary aufnehmen.

4: TA

Das, was wir eben getan haben, können wir leicht verallgemeinern, wenn wir "T" durch s ersetzen. Hierbei steht s wieder für  einen String, den wir im Dictionary finden können und den wir verlängern wollen.

Es gibt eine denkbare Schwierigkeit: Gerade haben wir nach der 3 die 0 gefunden, also einen Eintrag, den wir schon kennen. Was wäre nun, wenn in der Sequenz oben an zweiter Stelle eine 4 gestanden hätte, also wir das, was wir im Komprimierer gerade als neu hinzugefügt hatten, sofort auch tatsächlich finden?

Auf den ersten Blick scheint das ein Problem zu sein: woher soll der Expandierer nun wissen, welches Zeichen c der Komprimierer  eingelesen hatte, um es ans T zu hängen?

Überlegen wir mal: wissen wir wirklich nichts über c? Unsere Annahme ist, dass die Indexsequenz zu Beginn 3,4 enthält. Wie könnte das praktisch aussehen? Mit welchen Buchstabenkombinationen könnte der Text begonnen haben?

Mögliche 3er-Startsequenzen für unseren String, die mit T beginnen:

<pre>
TAA TAC TAG TAT 
TCA TCC TCG TCT
TGA TGC TGG TGT
TTA TTC TTG TTT
</pre>

Was würde der Komprimierer für diese Sequenzen als Ausgabe erzeugen:

<pre>
300 301 302 303
310 311 312 313
320 321 322 323
330 331 332 34
</pre>

Es kann hier nur die Sequenz TTT zu diesem Effekt führen! Und in diesem Fall wissen wir, dass das Zeichen c identisch sein muss mit dem ersten Zeichen von s (in unserem Fall: T). Das gilt allgemein! Erklären wir das:

Angenommen, unsere zuletzt ins Dictionary aufgenommene Sequence war $s_1 + c_1$, die Zeichenkette $s_1+c_1$ war uns also nicht bekannt. Ausgegeben haben wir zuletzt die Index-Nummer von $s_1$, $s_1$ ist uns also bekannt. 

Nun nehmen wir weiter an, dass wir als nächstes den Index von $s_1 + c_1$ ausgeben wollen (so, wie wir oben die Situation beschrieben haben). Wie muss dann unsere Textsequenz aussehen?

... $s_1$ [$c_1$] ...     : $s_1$ wird von $c_1$ gefolgt, das ist unbekannt für uns, $s_1+c_1$ wird aufgenommen

... $s_1$ $s_1$ $c_1$ ...   : jetzt geben wir den "neuen" Index von $s_1+c_1$ aus 

Das kann nur passieren, wenn das ERSTE ZEICHEN von s_1+c_1 gleich c_1 ist. Um nun also s_1 + c_1 in das Dictionary aufnehmen zu können, müssen wir an unser (uns ja bekanntes) s_1 einfach das erste Zeichen von s_1 anhängen!

Wenn wir also im Komprimierer auf die eben beschriebene Situation treffen, dann können wir an unser letztes s das erste Zeichen dieses s anhängen und erhalten so den Eintrag ins Dictionary, den wir brauchen, um weiter expandieren zu können.

Das berücksichtigen wir jetzt im folgenden Algorithmus (anders, als bei Cormen, speichern wir ins previous und current Strings und nicht Indexpositionen im Dictionary ab, nur relevant, wenn Sie die Lösungen miteinander vergleichen. Unsere Lösung ist einen Hauch kompakter).


In [30]:
def LZW_expand(seq):  # Text must be non-empty ;)
    # Init the dictionary
    dic = [ 'A', 'C', 'G', 'T']
    
    current = dic[seq[0]] # der erste String aus dem Dictionary, immer ein einzelner Buchstabe
    
    text = current # unser output momentan, nur der erste Buchstabe
    
    for i in seq[1:]:
        previous = current # was haben wir in der letzten Runde ausgegeben?
        if i == len(dic): # Unser Sonderfall...gerade aufgenommen und schon benutzt!
            word = previous+previous[0] 
        else:  # Das alte Wort plus der erste Buchstabe des nächsten Worts muss in dictionary
            word = previous+dic[i][0]
        dic.append(word) # Hier schreiben wir das Wort ins Dictionary
        current = dic[i] # Jetzt holen wir unsere monentane Ausgabe aus dem Dictionary       
        text += current  # Und hängen Sie an den Ausgabetext             
    return text

LZW_expand([3, 0, 4, 2, 5, 1, 3, 4, 5, 12])

'TATAGATCTTAATATA'

In [31]:
# Weitere Tests
print(LZW_expand(LZW("AGTGGGAATTAATTAGAGAGAGAGAGCCCCACACACT")))
print(LZW_expand(LZW("TTTTTTTTTTTTTTTTT")))

AGTGGGAATTAATTAGAGAGAGAGAGCCCCACACACT
TTTTTTTTTTTTTTTTT


In [41]:
# Und richtige Tests....
to_test = ["TTT","AGAGAGTTACCGCGCCCGCGCA","TTTTTTTTTTTTT","TATAGCCCGCGCGCGCGAA"]
for s in to_test:
    compression = LZW(s)
    expansion = LZW_expand(compression)
    print("Testing the compression and expansion of: \n\t",s,"\n\t",compression,"\n\t",expansion)
    if s == expansion:
        print("  Correct!")
    else:
        print("  ERROR!!!")

# Note, the functions above are not robust in any sense, don't try to compress strings with characters outside of
# the character set used in the functions! (But it is really easy to include the character set in the parameters!)

Testing the compression and expansion of: 
	 TTT 
	 [3, 4] 
	 TTT
  Correct!
Testing the compression and expansion of: 
	 AGAGAGTTACCGCGCCCGCGCA 
	 [0, 2, 4, 4, 3, 3, 0, 1, 1, 2, 12, 11, 14, 13, 0] 
	 AGAGAGTTACCGCGCCCGCGCA
  Correct!
Testing the compression and expansion of: 
	 TTTTTTTTTTTTT 
	 [3, 4, 5, 6, 5] 
	 TTTTTTTTTTTTT
  Correct!
Testing the compression and expansion of: 
	 TATAGCCCGCGCGCGCGAA 
	 [3, 0, 4, 2, 1, 8, 7, 10, 1, 10, 0, 0] 
	 TATAGCCCGCGCGCGCGAA
  Correct!


Auch für den Expansions-Algorithmus kann man einen Tabellenform finden, in der man den Ablauf des Algorithmus nachvollziehen kann:

<pre>
Schritt | in | previous | word  | Dictionary: | current 
 Init   |    |          |       |  0 : A      |
        |    |          |       |  1 : C      |
        |    |          |       |  2 : G      |
        |    |          |       |  3 : T      |
        | 3  |          |       |             |  T
   0    | 0  |  T       | TA    |  4 : TA     |  A
   1    | 4  |  A       | AT    |  5 : AT     |  TA
   2    | 2  |  TA      | TAG   |  6 : TAG    |  G
   3    | 5  |  G       | GA    |  7 : GA     |  AT
   4    | 1  |  AT      | ATC   |  8 : ATC    |  C
   5    | 3  |  C       | CT    |  9 : CT     |  T
   6    | 4  |  T       | TT    |  10 : TT    |  TA
   7    | 5  |  TA      | TAA   |  11 : TAA   |  AT
   8    | 12 |  AT      | ATA   |  12 : ATA   |  ATA
</pre>

TATAGATCTTAATATA


Die Spalte _current_ enthält die Ausgabe.

Aufgaben
--------

Denkbare Aufgaben:

Es wird jeweils eine initiales Dictionary vorgeben, z.B. so:

<pre>
0: A
1: B
2: C
3: D
4: E
</pre>

Dann kann man nach einer Kompression oder nach einer Expansion fragen:

(1) Gegeben eine Zeichenkette, die nur aus Buchstaben des vorgegebenen Alphabets besteht:

ABBCBABABDDEBCDE

Geben Sie eine Kompression an, also eine Sequenz von ganzen Zahlen.

(2) Gegeben eine Kompression, also eine Sequenz von ganzen Zahlen:

[ 2,3,1,5,4,6,3,2 ]

Geben Sie nun die Expansion dazu an.

Sie können jeweils selbst entscheiden, ob Sie die Tabellen verwenden wollen oder nicht. Wenn Sie die Tabellen NICHT verwenden und ihr Ergebnis nicht vollständig korrekt ist, kann ich keine Teilpunkte geben, ich werde nicht raten, wo sie den Fehler gemacht haben.

Hier der Vollständigkeit halber noch die Algorithmen von oben, die zusätzlich ein Alphabet als Eingabe erwarten und einen Parameter, den man auf True setzen kann, um die Tabellen auszugeben:


In [133]:
def LZW_all(text,dic,tabelle=True):  # Text must be non-empty ;)
    com = []  # Compression
    
    if tabelle: 
        step = 0
        print("\n\nSchritt |  c | s+c    | s       | Ausgabe | Dictionary:")
        for i in range(len(dic)):
            init = " Init" if i == 0 else "     "
            print(init,"  |    |        |         |         | ",i,":",dic[i])
        print("        |    |        |  \"\"     |         | ")

    s = ''
    for c in text:
        sc = s+c
        
        if sc in dic:  # ist s+c schon im Dictionary? 
            s += c      # Ja, dann warten wir, wie es weitergeht
            out = " "
            dic_out = ""
        else:           # Nein? Dann geben wir den Index von s aus, erweitern dic um s+c und setzen s auf c
            out = str(dic.index(s))
            com.append(dic.index(s)) # s must always be in dic! dic.index() gives the position of s in dic
            dic_out = ""+str(len(dic))+" : "+sc # preparing output
            dic.append(sc)
            s = c
        # Output   
        if tabelle:
            step_fill = "  " if step < 10 else " "
            out_fill  = "  " if len(out) < 2 else " "
            sc_fill = " "*(4-len(sc))
            s_fill = " "*(4-len(s))              
            print("  ",step,step_fill,"| ",c,"| ",sc,sc_fill,"| ",s,s_fill," |  ",out,out_fill," | ",dic_out)
            step += 1
        
    com.append(dic.index(s))
    if tabelle:
        step_fill = "  " if step < 10 else " "
        out = str(dic.index(s))
        out_fill  = "  " if len(out) < 2 else " "
        print("        |    |        |         |  ",out,out_fill," | \n")
    return com

print(LZW_all("ABBCBABABDDEBCDE",['A','B','C','D','E']))
print(LZW_all("TATAGATCTTAATATA", ['A','C','G','T']))

# Test a long one... ;)
# print(LZW_all("ABBCBABABDDEBCDEAABEBBEBDDBCCBDBDBBBDDCCCEAABABDEEBEBBBDDC",['A','B','C','D','E']))



Schritt |  c | s+c    | s       | Ausgabe | Dictionary:
 Init   |    |        |         |         |  0 : A
        |    |        |         |         |  1 : B
        |    |        |         |         |  2 : C
        |    |        |         |         |  3 : D
        |    |        |         |         |  4 : E
        |    |        |  ""     |         | 
   0    |  A |  A     |  A      |         |  
   1    |  B |  AB    |  B      |   0     |  5 : AB
   2    |  B |  BB    |  B      |   1     |  6 : BB
   3    |  C |  BC    |  C      |   1     |  7 : BC
   4    |  B |  CB    |  B      |   2     |  8 : CB
   5    |  A |  BA    |  A      |   1     |  9 : BA
   6    |  B |  AB    |  AB     |         |  
   7    |  A |  ABA   |  A      |   5     |  10 : ABA
   8    |  B |  AB    |  AB     |         |  
   9    |  D |  ABD   |  D      |   5     |  11 : ABD
   10   |  D |  DD    |  D      |   3     |  12 : DD
   11   |  E |  DE    |  E      |   3     |  13 : DE
   12   |  B |  EB    |  B    

In [131]:
def LZW_expand_all(seq,dic,tabelle=True):  # Text must be non-empty ;)
    
    if tabelle:
        print("\n\nSchritt | in | previous | word  | Dictionary: | current ")
        for i in range(len(dic)):
            init = " Init" if i == 0 else "     "
            print(init,"  |    |          |       | ",i,":",dic[i], "     |")


    
    current = dic[seq[0]] # der erste String aus dem Dictionary, immer ein einzelner Buchstabe
    if tabelle: print("        |",seq[0]," |          |       |             | ",current)
    
    text = current # unser output momentan, nur der erste Buchstabe
    
    step = 0
    for i in seq[1:]:
        previous = current # was haben wir in der letzten Runde ausgegeben?
        if i == len(dic): # Unser Sonderfall...gerade aufgenommen und schon benutzt!
            word = previous+previous[0] 
        else:  # Das alte Wort plus der erste Buchstabe des nächsten Worts muss in dictionary
            word = previous+dic[i][0]
        dic_out = ""+str(len(dic))+" : "+word # preparing output
        dic.append(word) # Hier schreiben wir das Wort ins Dictionary
        current = dic[i] # Jetzt holen wir unsere monentane Ausgabe aus dem Dictionary       
        text += current  # Und hängen Sie an den Ausgabetext     
        
        if tabelle: 
            step_fill = "  " if step < 10 else " "
            i_fill    = " " if i < 10 else ""
            prev_fill = " "*(4-len(previous))
            word_fill = " "*(4-len(word))
            dic_fill_cnt = 4-len(word)
            dic_fill_cnt -= 1 if len(dic) > 10 else 0
            dic_fill = " "*dic_fill_cnt
            print("  ",step,step_fill,"|",str(i)+i_fill,"| ",previous,prev_fill,"  |",word,word_fill,"| ",dic_out,dic_fill," | ",current)
            step += 1
    if tabelle: print("")  # Empty line
    return text

print(LZW_expand_all([3, 0, 4, 2, 5, 1, 3, 4, 5, 12],['A','C','G','T']))
print(LZW_expand_all([0, 1, 1, 2, 1, 5, 5, 3, 3, 4, 7, 13],['A','B','C','D','E']))

# nicht jede Sequenz von Zahlen geht!
print(LZW_expand_all([2,3,1,5,4,6,3,2],['A','B','C','D','E']))




Schritt | in | previous | word  | Dictionary: | current 
 Init   |    |          |       |  0 : A      |
        |    |          |       |  1 : C      |
        |    |          |       |  2 : G      |
        |    |          |       |  3 : T      |
        | 3  |          |       |             |  T
   0    | 0  |  T       | TA    |  4 : TA     |  A
   1    | 4  |  A       | AT    |  5 : AT     |  TA
   2    | 2  |  TA      | TAG   |  6 : TAG    |  G
   3    | 5  |  G       | GA    |  7 : GA     |  AT
   4    | 1  |  AT      | ATC   |  8 : ATC    |  C
   5    | 3  |  C       | CT    |  9 : CT     |  T
   6    | 4  |  T       | TT    |  10 : TT    |  TA
   7    | 5  |  TA      | TAA   |  11 : TAA   |  AT
   8    | 12 |  AT      | ATA   |  12 : ATA   |  ATA

TATAGATCTTAATATA


Schritt | in | previous | word  | Dictionary: | current 
 Init   |    |          |       |  0 : A      |
        |    |          |       |  1 : B      |
        |    |          |       |  2 : C      |
        |   

Selbst probieren
----------------

Wenn Sie die Algorithmen laufen lassen wollen, um selbst Aufgaben ausprobieren zu können, dann können sie

(1) Python 3 installieren und die Algorithmen in einem Editor eingeben und laufen lassen (zu Python gehört eine einfache IDE, dort können Sie es direkt eingeben): https://www.python.org/downloads/  (besser ist es, die Anaconda-Distribution zu verwenden, auch, wenn sie gar nicht die ipython/jupyter-Notebooks ausprobieren wollen: https://docs.continuum.io/anaconda/install oder https://conda.io/docs/install/quick.html (ohne das Schöne drumherum) bzw. https://www.continuum.io/downloads)

(2) Den Code nach repl.it kopieren und dort ausführen und modifizieren: https://repl.it/languages/python3

(3) Anaconda installieren, s. (1) (die beste Python-Distribution mit vielen weiteren Tools und Bibliotheken, z.B. numpy) und dort die jupyter-Notebooks ans Laufen bringen (ein lokaler Server auf ihrer eigenen Maschine), s. hier: http://jupyter.readthedocs.io/en/latest/install.html

(4) Sich einen Account bei https://www.kaggle.com erstellen, einen "Kernel" mit diesem ipython-Notebook, das sie bei Github direkt "raw" herunterladen können, erstellen.

(5) Sich einen Github-Account erstellen, das Notebook lokal ändern, nach Github pushen, dort ausführen und rendern lassen usw. Mühsam.

Die Lösung (3) ist am coolsten (weil man mit den Notebooks interaktiv sehr schön arbeiten kann), (1) und (2) sind auch gut, (4) ist auch 

Natürlich können sie die Algos auch nach Java portieren.